# 패키지 로딩

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score, precision_score

# 하이퍼 파라미터 설정

In [4]:
INPUT_DIMS = 13 # 13개의 독립변수
EPOCH = 100
BATCH = 32

# 데이터 로딩 및 확인

In [8]:
!pip install xlrd

   ---------------------------------------- 0.0/96.5 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/96.5 kB ? eta -:--:--
   ------------------------- -------------- 61.4/96.5 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 96.5/96.5 kB 1.4 MB/s eta 0:00:00


DEPRECATION: mecab-python 0.996-ko-0.9.2-msvc has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of mecab-python or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [9]:
data = pd.read_excel('heart.xls')
print(data.shape)
print(data.head()) # target값을 제외하면 독립변수 13개

(303, 14)
   age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  slope  \
0   63    1   3       145   233    1        0      150      0      2.3      0   
1   37    1   2       130   250    0        1      187      0      3.5      0   
2   41    0   1       130   204    0        0      172      0      1.4      2   
3   56    1   1       120   236    0        1      178      0      0.8      2   
4   57    0   0       120   354    0        1      163      1      0.6      2   

   ca  thal  target  
0   0     1       1  
1   0     2       1  
2   0     2       1  
3   0     2       1  
4   0     2       1  


In [10]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   fbs       303 non-null    int64  
 6   restecg   303 non-null    int64  
 7   thalach   303 non-null    int64  
 8   exang     303 non-null    int64  
 9   oldpeak   303 non-null    float64
 10  slope     303 non-null    int64  
 11  ca        303 non-null    int64  
 12  thal      303 non-null    int64  
 13  target    303 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 33.3 KB
None


In [11]:
data.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


# 데이터 스케일링

In [12]:
from sklearn.preprocessing import StandardScaler

x = data.drop('target', axis = 1)
y = data['target']
scaler = StandardScaler()
scaled_x = scaler.fit_transform(x)

# 학습/검증/평가 데이터 분할

In [15]:
x_train, x_test, y_train, y_test = train_test_split(scaled_x, y, train_size=0.8, random_state=10)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((242, 13), (61, 13), (242,), (61,))

In [17]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.8, random_state=0)
x_train.shape, x_val.shape, y_train.shape, y_val.shape

((193, 13), (49, 13), (193,), (49,))

# 모델 생성

In [18]:
from keras.layers import Dropout
from keras import regularizers

model = Sequential()
model.add(Dense(units=1000, activation='tanh', input_dim=INPUT_DIMS, kernel_regularizer=regularizers.L2(0.02))) # units : 히든 레이어의 출력 데이터 개수, 0.02 : 규제 강도
model.add(Dense(units=1000, activation='tanh', kernel_regularizer=regularizers.L2(0.1)))
model.add(Dropout(rate=0.5))
model.add(Dense(units=1, activation='sigmoid'))
print(model.summary())

C:\Users\user\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 1000)                │          14,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1000)                │       1,001,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1000)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │           1,001 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,016,001 (3.88 MB)

 Trainable params: 1,016,001 (3.88 MB)

 Non-trainable params: 0 (0.00 B)

None


# 모델 컴파일 및 학습

## Tensorboard 사용 준비

In [22]:
from keras.callbacks import TensorBoard
import datetime

log_dir = 'c:\\Logs\\' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
# histogram_freq = 1 : 1 에포크마다 활성화 출력의 히스토그램을 기록 
tensorboard = TensorBoard(log_dir=log_dir, histogram_freq = 1)

## EarlyStopping 설정
- EarlyStopping 매개변수
>- monitor: 모니터링할 지표를 선택. 'val_loss', 'val_acc' 등의 값을 지정
>- patience: 모니터링 지표의 값에 개선이 없다고 판단하기 전에 대기할 epoch 수
>- mode : 모니터링할 지표를 최소화할지 최대화할지 결정. 'min': 최소화, 'max':최대

In [23]:
from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', mode='min', patience=3) # 값이 최소가 된 이후로 3번의 반복을 더 진행하고 변화가 없으면 조기 종료

In [25]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=BATCH, epochs=EPOCH, validation_data=(x_val, y_val), verbose=1, callbacks=[tensorboard, early_stop])

Epoch 1/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - accuracy: 0.6885 - loss: 83.0652 - val_accuracy: 0.5306 - val_loss: 48.3964
Epoch 2/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.6708 - loss: 43.0642 - val_accuracy: 0.5102 - val_loss: 29.3338
Epoch 3/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.6880 - loss: 26.2161 - val_accuracy: 0.8367 - val_loss: 17.6349
Epoch 4/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.7963 - loss: 15.7958 - val_accuracy: 0.7959 - val_loss: 10.5385
Epoch 5/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.8004 - loss: 9.3589 - val_accuracy: 0.8571 - val_loss: 5.9272
Epoch 6/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.8497 - loss: 5.2142 - val_accuracy: 0.8776 - val_loss: 3.1412
Epoch 7/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.8529 - loss: 2.7656 - val_accuracy: 0.8163 - val_loss: 1.8088
Epoch 8/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.8485 - loss: 1.5794 - val_accuracy: 0.5306 - v

# 예측 및 모델 평가

In [27]:
y_pred_prob = model.predict(x_test)
# print(y_pred_prob[:5])
y_pred = (y_pred_prob > 0.5)
print(y_pred[:5])

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
[[False]
 [ True]
 [False]
 [ True]
 [False]]


In [28]:
print('Confusion Matrix')
print(confusion_matrix(y_test, y_pred))

Confusion Matrix
[[25 10]
 [ 4 22]]


In [34]:
score = model.evaluate(x_test, y_test)
print(f'Loss: {score[0]:.3f}')
print(f'Accuracy: {score[1]:.3f}')
print(f'Precision : {precision_score(y_test, y_pred):.3f}')
print(f'AUC:{roc_auc_score(y_test, y_pred_prob):.3f}')

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.7741 - loss: 0.6929  
Loss: 0.698
Accuracy: 0.770
Precision : 0.688
AUC:0.869
